# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,22.80,92,75,2.57,US,1715234328
1,1,blackmans bay,-43.0167,147.3167,13.18,93,84,1.29,AU,1715234162
2,2,harstad,68.7990,16.5293,5.66,83,100,8.94,NO,1715234328
3,3,edinburgh of the seven seas,-37.0676,-12.3116,14.38,68,100,13.19,SH,1715234328
4,4,el colomo,19.0500,-104.2500,21.34,87,43,1.37,MX,1715234328


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"
)
# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[city_data_df['Wind Speed'] <= 7]

# Drop any rows with null values
ideal_city_df.dropna(how='any', axis=0)

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hawaiian paradise park,19.5933,-154.9731,22.80,92,75,2.57,US,1715234328
1,1,blackmans bay,-43.0167,147.3167,13.18,93,84,1.29,AU,1715234162
4,4,el colomo,19.0500,-104.2500,21.34,87,43,1.37,MX,1715234328
5,5,adamstown,-25.0660,-130.1015,21.79,78,69,3.22,PN,1715234149
7,7,grytviken,-54.2811,-36.5092,-3.56,69,8,2.51,GS,1715234158
...,...,...,...,...,...,...,...,...,...,...
556,556,akhmet'a,42.0311,45.2075,14.29,81,100,1.63,GE,1715234442
557,557,xinyuan,37.3000,99.0000,15.52,22,100,5.48,CN,1715234442
558,558,valdivia,-39.8142,-73.2459,8.05,100,99,0.88,CL,1715234443
559,559,sorong,-0.8833,131.2500,29.20,69,100,1.51,ID,1715234443


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = ideal_city_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hawaiian paradise park,US,19.5933,-154.9731,92,
1,blackmans bay,AU,-43.0167,147.3167,93,
4,el colomo,MX,19.0500,-104.2500,87,
5,adamstown,PN,-25.0660,-130.1015,78,
7,grytviken,GS,-54.2811,-36.5092,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
limit = 1
radius = 10000
hotel = "accommodation.hotel"
f"circle:{long},{lati},{radius}"
f"proximity:{long},{lati}"

params = {
    "limit":limit,
    "apiKey":geoapify_key,
    "categories":hotel
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lati = hotel_df.loc[index,"Lat"]
    long = hotel_df.loc[index,"Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lati},{radius}"
    params["bias"] = f"proximity:{long},{lati}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    #print(name_address.content)

    #print(name_address)
    # Convert the API response to JSON format
    name_address = name_address.json()

    #print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
el colomo - nearest hotel: Hotel Marina Puerto Dorado
adamstown - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
alto araguaia - nearest hotel: Hotel Araguaia
turbat - nearest hotel: No hotel found
bredasdorp - nearest hotel: Victoria Hotel
robore - nearest hotel: Lajas
taltal - nearest hotel: Hostería Taltal
bamboo flat - nearest hotel: Hotel Megapode Nest
rio grande - nearest hotel: Hotel Vila Moura Executivo
flying fish cove - nearest hotel: Christmas Island Lodge
keflavik - nearest hotel: Núpan Deluxe
waitangi - nearest hotel: Hotel Chathams
la'ie - nearest hotel: No hotel found
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
tiksi - nearest hotel: Арктика
dolinsk - nearest hotel: Юбилейная
saldanha - nearest hotel: Hoedjesbaai
tado - nearest hotel: Hotel Las M.M.M.
emerald - nearest hotel: Emerald Central Hotel
talnakh - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
0,hawaiian paradise park,US,19.5933,-154.9731,92,No hotel found
1,blackmans bay,AU,-43.0167,147.3167,93,Villa Howden
4,el colomo,MX,19.0500,-104.2500,87,Hotel Marina Puerto Dorado
5,adamstown,PN,-25.0660,-130.1015,78,No hotel found
7,grytviken,GS,-54.2811,-36.5092,69,No hotel found
...,...,...,...,...,...,...
556,akhmet'a,GE,42.0311,45.2075,81,სასტუმრო
557,xinyuan,CN,37.3000,99.0000,22,No hotel found
558,valdivia,CL,-39.8142,-73.2459,100,Esmeralda
559,sorong,ID,-0.8833,131.2500,69,Marina Mamberamo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ['Country','Hotel Name']
    
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)